# Exercise Introduction
We will return to the automatic rotation problem you worked on in the previous exercise. But we'll add data augmentation to improve your model.

The model specification and compilation steps don't change when you start using data augmentation. The code you've already worked with for specifying and compiling a model is in the cell below.  Run it so you'll be ready to work on data augmentation.

In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_5 import *
print("Setup Complete")

/Users/danbecker/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Setup Complete


# 1) Fit the Model Using Data Augmentation

Here is some code to set up some ImageDataGenerators. Run it, and then answer the questions below about it.

In [2]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224

# Specify the values for all arguments to data_generator_with_aug.
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True,
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)
            
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)


Why do we need both a generator with augmentation and a generator without augmentation? After thinking about it, check out the solution below.

In [3]:
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
We want to do data augmentation when fitting the model for the reasons mentioned
in the video (including a reduction in overfitting, by giving us more data to work with).

But we don't want to change how we test the model. So the validation generator
will use an ImageDataGenerator without augmentation. That allows a straightforward
comparison between different training procedures (e.g. training with augmentation
and without it).

If the augmentation made it harder to predict the label associated with an image
(e.g. because of how the image was cropped in augmentation) that would make it misleading
to compare scores to another procedure where the validation data was only original images.


# 2) Choosing Augmentation Types
ImageDataGenerator offers many types of data augmentation. For example, one argument is `rotation_range`. This rotates each image by a random amount that can be up to whatever value you specify.

Would it be sensible to use automatic rotation for this problem?  Why or why not?

In [4]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
The goal in this problem is to tell if an image is upright or sideways.  Rotating images
moderately might cause images that don't feel cleanly in either category.

Since data augmentation affects images without touching the labels, a dramatic rotation
would mean some images are used for training with the wrong label (e.g. an image would be rotated
sideways by the generator, and still have a label of being upright)


# 3) Code
Fill in the missing pieces in the following code. We've supplied some boilerplate. You need to think about what ImageDataGenerator is used for each data source.

In [5]:
# Specify which type of ImageDataGenerator above is to load in training data
train_generator = data_generator_with_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/train',
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical')

# Specify which type of ImageDataGenerator above is to load in validation data
validation_generator = data_generator_no_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator, # specify where model gets training data
        epochs = 3,
        steps_per_epoch=19,
        validation_data=validation_generator) # specify where model gets validation data

q_3.check()

Found 220 images belonging to 2 classes.
Found 217 images belonging to 2 classes.
Epoch 1/3
19/19 [==============================] - 57s 3s/step - loss: 0.6237 - acc: 0.6977 - val_loss: 0.2428 - val_acc: 0.9124
Epoch 2/3
19/19 [==============================] - 54s 3s/step - loss: 0.2280 - acc: 0.9253 - val_loss: 0.1978 - val_acc: 0.9447
Epoch 3/3
19/19 [==============================] - 53s 3s/step - loss: 0.1466 - acc: 0.9692 - val_loss: 0.1566 - val_acc: 0.9631


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# q_3.solution()

# 4) Did Data Augmentation Help?
How could you test whether data augmentation improved your model accuracy?

In [7]:
q_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
Create `train_generator` usng `data_generator_no_aug` but don't
change other arguments to `train_generator`.

Run the model and see the resuling accuracy. Compare this to
the accuracy you got when `train_generator` used augmentation.

Our validation dataset is very small, so there's a little bit
of luck or randomness in the exact score from any
model run. Validation scores will be more reliable as you
start using larger datasets.


# Keep Going
You are ready for **[a deeper understanding of deep learning](https://www.kaggle.com/dansbecker/a-deeper-understanding-of-deep-learning/)**.